In [1]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Set the chromedriver path
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/95.0.4638.54/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/yol/.wdm/drivers/chromedriver/mac64/95.0.4638.54]


In [3]:
# Load the Mars news site
news_url = 'https://redplanetscience.com'
browser.visit(news_url)

In [4]:
# Get the latest news headline and teaser
news_html = browser.html
news_soup = BeautifulSoup(news_html, 'html.parser')

date = news_soup.find('div', {'class': 'list_date'}).text
title = news_soup.find('div', {'class': 'content_title'}).text
teaser = news_soup.find('div', {'class': 'article_teaser_body'}).text

if (date and title and teaser):
    print(date)
    print(title)
    print(teaser)

November 5, 2021
NASA InSight's 'Mole' Is Out of Sight
Now that the heat probe is just below the Martian surface, InSight's arm will scoop some additional soil on top to help it keep digging so it can take Mars' temperature.


In [5]:
# Load the Mars images site
images_url = 'https://spaceimages-mars.com'
browser.visit(images_url)

In [6]:
# Get the featured image url
images_html = browser.html
images_soup = BeautifulSoup(images_html, 'html.parser')

images_path = images_soup.find('img', {'class': 'headerimage fade-in'}).get('src')
featured_image_url = f'{images_url}/{images_path}'
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars3.jpg


In [7]:
# Load the Mars facts site
facts_url = 'https://galaxyfacts-mars.com'
browser.visit(facts_url)

In [8]:
# Scrape the Mars facts table 
tables = pd.read_html(facts_url)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [9]:
# Dump it into a dataframe
mars_facts_df = tables[0]
mars_facts_df.columns = mars_facts_df.iloc[0]
mars_facts_df = mars_facts_df.iloc[1:].reset_index(drop=True)
mars_facts_df

,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days
5,Temperature:,-87 to -5 °C,-88 to 58°C


In [10]:
# Save it to an html file
mars_facts_df.to_html('mars_facts.html', index=False, index_names=False)
!open mars_facts.html

In [11]:
# Load the Mars hemispheres site
hemisphere_url = 'https://marshemispheres.com'
browser.visit(hemisphere_url)

In [15]:
hemisphere_html = browser.html
hemisphere_soup = BeautifulSoup(hemisphere_html,'html.parser')
print(hemisphere_soup.prettify())

<html lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="css/jquery-ui.css" rel="stylesheet" type="text/css"/>
  <title>
   Astropedia Search Results | GUSS Astrogeology Science Center
  </title>
  <meta content="GUSS Astrogeology Science Center Astropedia search results." name="description"/>
  <meta content="GUSS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
  <link href="css/main.css" media="screen" rel="stylesheet"/>
  <link href="css/print.css" media="print" rel="stylesheet"/>
  <link href="#" rel="icon" type="image/x-ico"/>
 </head>
 <body id="results">
  <header>
   <a href="#" style="float:right;margin-top:10px;" target="_blank">
    <img alt="USGS: Science for a Changing World" class="logo" height="60" src="images/usgs_logo_main_

In [24]:
results = hemisphere_soup.find_all('Hemisphere').text
print(results)

AttributeError: ResultSet object has no attribute 'text'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?